In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from notebooks.imports import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
import warnings
# suppress warnings
warnings.filterwarnings("ignore")

import pickle
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import statsmodels.stats.api as sms
from sklearn import preprocessing

import src.utils.setup_figures as figures
# import src.utils.basic as utils
from src.utils.glm_hmm_utils import global_fit, session_wise_fit_cv


import numpy.random as npr
# npr.seed(0)

### Load Configs

In [14]:
from config import dir_config, main_config

raw_dir = Path(dir_config.data.raw)
processed_dir = Path(dir_config.data.processed)
compiled_dir = Path(dir_config.data.compiled)

### Utils functions

In [15]:
def process_session_data(data):
    data['target'] = (2*data.target - 1).astype(int)
    data['choice'] = data['choice'].astype(int)
    data['signed_coherence'] = data['coherence'] * data['target']
    data['previous_target'] = data.target.shift(1)
    data.loc[data.index[0], 'previous_target'] = data.target.values[0]
    data['previous_choice'] = data.choice.shift(1)
    data.loc[data.index[0], 'previous_choice'] = data.choice.values[0]
    
    first_valid = np.where(data.outcome >= 0)[0][0]
    indices_to_change_previous_choice = data[data.previous_choice == -1].index
        
    for i in indices_to_change_previous_choice:
        if i < first_valid:
            data.loc[data.index[i], 'previous_choice'] = np.random.choice([-1, 1])
            data.loc[data.index[i], 'previous_target'] = np.random.choice([-1, 1])
        else:
            j=0
            while i - j >= 0 and data.at[data.index[i - j], 'previous_choice'] == -1:
                j+=1
            data.loc[data.index[i], 'previous_choice'] = data.loc[data.index[i-j], 'previous_choice']
            data.loc[data.index[i], 'previous_target'] = data.loc[data.index[i-j], 'previous_target']  
    
    
    data['previous_choice'].replace(0, -1, inplace=True) # remap previous choice to -1, 1
    
    return data

def extract_previous_data(trial_data):
    # npr.seed()
    prev_choice = np.hstack([trial_data.choice[0] , trial_data.choice[:-1]])  # 0:awayRF, 1:toRF of previous valid trial
    prev_target = np.hstack([trial_data.target[0] , trial_data.target[:-1]]) * 2 - 1 # -1:awayRF, 1:toRF of previous valid trial 
    prev_outcome = np.hstack([trial_data.outcome[0] , trial_data.outcome[:-1]])
    prev_valid_idx = np.where(prev_outcome >= 0)[0]
    prev_invalid_idx = np.where(prev_outcome == -1)[0]

    for i in prev_invalid_idx:
        if i < prev_valid_idx[0]: #randomly sample if no previous valid trials
            prev_choice[i] = np.random.binomial(1,0.5)
            prev_target[i] = np.random.binomial(1,0.5) * 2 - 1
        else:
            last_valid =  np.where(prev_valid_idx<i)[0][-1]
            prev_choice[i] = prev_choice[prev_valid_idx[last_valid]]
            prev_target[i] = prev_target[prev_valid_idx[last_valid]]
            
    prev_choice = (prev_choice * 2) - 1 # -1:awayRF, 1:toRF of previous valid trial
    return prev_choice.astype(int), prev_target.astype(int)

def prepare_input_data(data, input_dim):
    X = np.ones((1, data.shape[0], input_dim))
    
    current_stimulus = data.coherence * (2*data.target-1)
    current_stimulus = current_stimulus / 100
    
    X[0,:,0] = current_stimulus
    X[0,:,2], X[0,:,3] = extract_previous_data(data)
    return list(X)
    

### Load Data

In [16]:
session_metadata = pd.read_csv(Path(compiled_dir, "sessions_metadata.csv"), index_col=None)

#### Create design matrix (input, observation, mask)

In [17]:
n_states = 2            # number of discrete states
obs_dim = 1             # number of observed dimensions: choice
num_categories = 2      # number of categories for output (toRF/awayRF)
input_dim = 4           # input dimensions: current signed coherence, 1(bias), previous choice(toRF/awayRF), previous target side(toRF/awayRF)

#### Data preparation

In [18]:
inputs_session_wise = []
choices_session_wise = []
invalid_idx_session_wise = []
masks_session_wise = []

for session_id in (session_metadata['session_id']):
    session_data = pd.read_csv(Path(compiled_dir, session_id, f"{session_id}_trial.csv"), index_col=None).fillna(-1)
    gp_trial_data = session_data[session_data.task_type == 1].reset_index()

    inputs_session_wise += prepare_input_data(gp_trial_data, input_dim)
    choices = gp_trial_data.choice.values.reshape(-1,1).astype('int')

    masks_session_wise.append(np.array(choices >= 0))
    invalid_idx = np.where(choices == -1)[0].reshape(-1,1)
    invalid_idx_session_wise.append(invalid_idx)

    # for training, replace -1 with random sample from 0,1
    choices[choices == -1] = npr.choice(1,invalid_idx.shape[0])
    choices_session_wise.append(choices)
    
inputs_aggregated, choices_aggregated, masks_aggregated = [], [], []
inputs_aggregated.append(np.vstack(inputs_session_wise))
choices_aggregated.append(np.vstack(choices_session_wise))
masks_aggregated.append(np.vstack(masks_session_wise))

# scaling signed coherence
inputs_aggregated[0][:,0] = preprocessing.scale(inputs_aggregated[0][:, 0], axis=0)
for idx_session in range(len(session_metadata)):
    inputs_session_wise[idx_session][:,0] = preprocessing.scale(inputs_session_wise[idx_session][:, 0], axis=0) # normalize signed coherence


### Fit data

In [127]:
models_glm_hmm, fit_lls_glm_hmm = global_fit(observations=choices_aggregated, inputs=inputs_aggregated, masks=masks_aggregated, n_iters=2000, n_initializations=20)

Fitting GLM globally...


  0%|          | 0/2000 [00:00<?, ?it/s]

Fitting 2 states...


Converged to LP: -22541.4:  42%|████▏     | 833/2000 [04:54<06:52,  2.83it/s]


Fitting 3 states...


Converged to LP: -22519.6:  86%|████████▌ | 1716/2000 [09:24<01:33,  3.04it/s]


Fitting 4 states...


LP: -22539.3: 100%|█████████▉| 1999/2000 [20:12<00:00,  4.54it/s]

Fitting 5 states...


LP: -21252.9: 100%|██████████| 2000/2000 [17:03<00:00,  1.95it/s]


In [144]:
global_fits = {
    'models': models_glm_hmm,
    'fits_lls_glm_hmm': fit_lls_glm_hmm,
}

with open(Path(processed_dir, 'models_glm_hmm_global.pkl'), 'wb') as f:
    pickle.dump(global_fits, f)

In [128]:
# get best model of 20 initializations for each state
init_params = {
    'glm_weights': {},
    'transition_matrices': {}
}
for n_states in np.arange(2,6):
    best_idx = np.argmax([np.max(arr) for arr in fit_lls_glm_hmm[n_states]])
    init_params['glm_weights'][n_states] = models_glm_hmm[n_states][best_idx].observations.params
    init_params['transition_matrices'][n_states] = models_glm_hmm[n_states][best_idx].transitions.params

In [141]:
# fit session-wise with 5 fold cross-validation
models_session_state_fold, session_train_ll, session_test_ll = session_wise_fit_cv(observations=choices_session_wise, inputs=inputs_session_wise, masks=masks_session_wise,
                                                                    n_sessions=len(session_metadata), init_params=init_params, n_iters=2000)

Fitting session 0...
Fitting 2 states...


Converged to LP: -378.5:   1%|          | 13/2000 [00:00<00:24, 80.03it/s]


Fitting 3 states...


Converged to LP: -397.2:   3%|▎         | 60/2000 [00:00<00:19, 97.36it/s] 


Fitting 4 states...


...

Converged to LP: -441.8:   1%|          | 11/2000 [00:00<00:27, 71.32it/s]


Fitting 4 states...


Converged to LP: -462.9:   3%|▎         | 52/2000 [00:00<00:21, 90.12it/s]


Fitting 5 states...


Converged to LP: -484.9:   9%|▊         | 172/2000 [00:02<00:21, 83.24it/s] 


In [145]:
session_wise_fits = {
    'models': models_session_state_fold,
    'train_ll': session_train_ll,
    'test_ll': session_test_ll
}

with open(Path(processed_dir, 'models_glm_hmm_session_wise.pkl'), 'wb') as f:
    pickle.dump(session_wise_fits, f)
    